In [243]:
from IPython.display import display, Markdown
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer, KNNImputer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler , OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
import math
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import pandas as pd 
import scipy.stats as ss
import seaborn as sns
import os
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Decide between local or kaggle cloud storage         
KAGGLE_ENV = 'kaggle' in os.listdir('/')
data_path = '/kaggle/input' if KAGGLE_ENV else '../kaggle/input'

# This is a good idea to work only locally. But If you wanna ran your NB also at kaggle... this is not working.
# # Pull the dataset from kaggle, it is concat dataset train + original dataset
# dataset_name = 'dantheshark/s4-e11-train-concat'
# if KAGGLE_ENV:
#     kaggle.api.dataset_download_files(dataset_name, path="../kaggle/input/", unzip=True)


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
    
for dirname, _, filenames in os.walk(data_path):
    for filename in filenames:
        print(os.path.join(dirname, filename)) 

../kaggle/input/submission.csv
../kaggle/input/depression-surveydataset-for-analysis/final_depression_dataset_1.csv
../kaggle/input/playground-series-s4e11/sample_submission.csv
../kaggle/input/playground-series-s4e11/test.csv
../kaggle/input/playground-series-s4e11/train.csv
../kaggle/input/s4-e11-train-concat-final/s4-e11-train-concat-final.csv
../kaggle/input/s4-e11-train-concat-final/dataset-metadata.json
../kaggle/input/s4-e11-train-concat/s4-e11-train-concat.csv


# Load Data

In [244]:
# Load the data
train_original = pd.read_csv(data_path + '/playground-series-s4e11/train.csv')
train_original = pd.read_csv(data_path + '/playground-series-s4e11/test.csv')
sample_submission = pd.read_csv(data_path + '/playground-series-s4e11/sample_submission.csv')
original_data = pd.read_csv(data_path + '/depression-surveydataset-for-analysis/final_depression_dataset_1.csv')
train_concat_data = pd.read_csv(data_path + '/../working/s4-e11-train-concat1.csv')

# Preprocessing

## Convertion from Features

In [245]:
# Convert Float to Int, not needed float
columns_to_convert = ['Work Pressure', 'Job Satisfaction', 'Study Satisfaction', 'Work/Study Hours', 'Financial Stress']

for col in columns_to_convert:
    train_concat_data[col] = pd.to_numeric(train_concat_data[col], errors='coerce').astype('Int64')

In [246]:
def get_categorical_numerical_features(df):
    # Get Numeric & Categorical Features
    numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
    categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()
    return numeric_features, categorical_features
numeric_features, categorical_features = get_categorical_numerical_features(train_concat_data)

## Outliers

In [247]:
# plt.figure(figsize=(12, 6))
# sns.boxplot(data=train_concat_data[numeric_features])
# plt.xticks(rotation=90)
# plt.show()

## Show general stats 

In [248]:
def show_general_stats(df):
    display(Markdown('### General Stats'))
    display(df.describe())
    display(Markdown('### Data Types'))
    display(df.dtypes)
    display(Markdown('### Missing Values'))
    display(df.isnull().sum())
    display(Markdown('### Shape'))
    display(df.shape)
    display(Markdown('### Head'))
    display(df.head(100))
    display(Markdown('### Tail'))
    display(df.tail(100))
    display(Markdown('### Sample'))
    display(df.sample(100))
    display(Markdown('### '))

In [249]:
show_general_stats(train_concat_data)

### General Stats

,Work Pressure,Study Satisfaction,Job Satisfaction,Work/Study Hours,Financial Stress,Depression
count,114836.0,28399.0,114844.0,143256.0,143252.0,143256.000000
mean,2.999408,2.947252,2.975131,6.248597,2.988621,0.181647
std,1.405975,1.360518,1.416124,3.852275,1.413664,0.385555
min,1.0,1.0,1.0,0.0,1.0,0.000000
25%,2.0,2.0,2.0,3.0,2.0,0.000000
50%,3.0,3.0,3.0,6.0,3.0,0.000000
75%,4.0,4.0,4.0,10.0,4.0,0.000000
max,5.0,5.0,5.0,12.0,5.0,1.000000


### Data Types

Working Professional or Student          object
Work Pressure                             Int64
Study Satisfaction                        Int64
Job Satisfaction                          Int64
Degree                                   object
Have you ever had suicidal thoughts ?    object
Work/Study Hours                          Int64
Financial Stress                          Int64
Depression                                int64
dtype: object

### Missing Values

...

### Tail

,Working Professional or Student,Work Pressure,Study Satisfaction,Job Satisfaction,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Depression
143156,Working Professional,3,<NA>,3,BSc,No,10,5,0
143157,Working Professional,1,<NA>,5,BSc,No,5,4,0
143158,Working Professional,2,<NA>,4,M.Pharm,Yes,1,4,0
143159,Working Professional,4,<NA>,1,Other,Yes,2,5,1
143160,Student,<NA>,3,<NA>,Class 12,Yes,6,5,1
...,...,...,...,...,...,...,...,...,...
143251,Working Professional,1,<NA>,5,BBA,Yes,12,3,0
143252,Working Professional,3,<NA>,1,Other,Yes,8,3,1
143253,Working Professional,1,<NA>,4,Other,Yes,4,4,0
143254,Working Professional,2,<NA>,3,BBA,No,4,5,0


### Sample

,Working Professional or Student,Work Pressure,Study Satisfaction,Job Satisfaction,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Depression
83201,Working Professional,1,<NA>,3,Other,No,0,4,0
55890,Working Professional,3,<NA>,2,Other,Yes,9,3,0
141434,Working Professional,4,<NA>,5,Other,No,2,4,0
114178,Working Professional,2,<NA>,1,Other,No,7,3,0
77613,Working Professional,1,<NA>,4,BBA,Yes,5,2,0
...,...,...,...,...,...,...,...,...,...
117382,Working Professional,3,<NA>,3,B.Com,Yes,6,2,0
94462,Working Professional,1,<NA>,4,B.Pharm,No,7,4,0
11190,Working Professional,2,<NA>,1,Class 12,Yes,12,5,1
64277,Working Professional,2,<NA>,3,MCA,Yes,11,5,0


### 

## MICE (Multiple Imputation by Chained Equations)
**We don't use MICE**, at the train data there is not enough correlation between the features (none of them > +/- 0.7!)

In [250]:
# def cramers_v(x, y):
#     confusion_matrix = pd.crosstab(x, y)
#     chi2 = ss.chi2_contingency(confusion_matrix)[0]
#     n = confusion_matrix.sum().sum()
#     phi2 = chi2 / n
#     r, k = confusion_matrix.shape
#     phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
#     rcorr = r - ((r-1)**2)/(n-1)
#     kcorr = k - ((k-1)**2)/(n-1)
#     return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

# cramers_v(train_concat_data['FEATURE'], train_concat_data['Depression'])

## KNN Imputer (k-Nearest Neighbors)
**Takes way too long for this dataset**

In [251]:
# imputer = KNNImputer(n_neighbors=3)  # k=5 nearest neighbors
# print("Start KNN-Imputation...")
# train_original[numeric_features] = imputer.fit_transform(train_original[numeric_features])# only for numerical data
# print(train_original)

## Missing Data Numerical (int), IterativeImputer (estimator=RandomForestClassifier) 
Tyred also ExtraTreesRegressor, but does not converged.

In [252]:
# List of columns (only int) with missing values
columns_to_convert = ['Work Pressure', 'Job Satisfaction',  'Work/Study Hours', 'Financial Stress']#, 'Study Satisfaction']# Had problems here, messed up the data 

train_preprocessed = train_concat_data.copy()

imputer = IterativeImputer(
    estimator=RandomForestClassifier(n_estimators=50, max_depth=10, n_jobs=-1, random_state=42),
    max_iter=200,
    random_state=42,
    min_value=0,
    imputation_order="ascending" 
)
df_subset = train_preprocessed[columns_to_convert]
df_imputed_values = imputer.fit_transform(df_subset)
train_preprocessed[columns_to_convert] = np.round(df_imputed_values).astype(int)

print(imputer.n_iter_)
show_general_stats(train_preprocessed)

4


### General Stats

,Work Pressure,Study Satisfaction,Job Satisfaction,Work/Study Hours,Financial Stress,Depression
count,143256.000000,28399.0,143256.000000,143256.000000,143256.000000,143256.000000
mean,3.031049,2.947252,2.850561,6.248597,2.988657,0.181647
std,1.420301,1.360518,1.422842,3.852275,1.413677,0.385555
min,1.000000,1.0,1.000000,0.000000,1.000000,0.000000
25%,2.000000,2.0,2.000000,3.000000,2.000000,0.000000
50%,3.000000,3.0,3.000000,6.000000,3.000000,0.000000
75%,4.000000,4.0,4.000000,10.000000,4.000000,0.000000
max,5.000000,5.0,5.000000,12.000000,5.000000,1.000000


### Data Types

Working Professional or Student          object
Work Pressure                             int64
Study Satisfaction                        Int64
Job Satisfaction                          int64
Degree                                   object
Have you ever had suicidal thoughts ?    object
Work/Study Hours                          int64
Financial Stress                          int64
Depression                                int64
dtype: object

...

### Tail

,Working Professional or Student,Work Pressure,Study Satisfaction,Job Satisfaction,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Depression
143156,Working Professional,3,<NA>,3,BSc,No,10,5,0
143157,Working Professional,1,<NA>,5,BSc,No,5,4,0
143158,Working Professional,2,<NA>,4,M.Pharm,Yes,1,4,0
143159,Working Professional,4,<NA>,1,Other,Yes,2,5,1
143160,Student,2,3,5,Class 12,Yes,6,5,1
...,...,...,...,...,...,...,...,...,...
143251,Working Professional,1,<NA>,5,BBA,Yes,12,3,0
143252,Working Professional,3,<NA>,1,Other,Yes,8,3,1
143253,Working Professional,1,<NA>,4,Other,Yes,4,4,0
143254,Working Professional,2,<NA>,3,BBA,No,4,5,0


### Sample

,Working Professional or Student,Work Pressure,Study Satisfaction,Job Satisfaction,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Depression
101908,Working Professional,1,<NA>,1,BBA,Yes,5,4,0
136223,Working Professional,2,<NA>,1,BSc,Yes,6,4,0
78196,Working Professional,3,<NA>,5,BCA,Yes,8,2,0
5789,Working Professional,5,<NA>,1,MCA,Yes,1,4,1
72792,Working Professional,4,<NA>,5,Other,Yes,5,2,0
...,...,...,...,...,...,...,...,...,...
23649,Working Professional,1,<NA>,5,LLM,No,9,5,0
68465,Working Professional,3,<NA>,2,Other,No,4,3,0
72588,Working Professional,4,<NA>,5,Other,No,2,1,0
135316,Working Professional,5,<NA>,3,LLM,Yes,1,3,0


### 

In [253]:
columns_to_convert = ['Study Satisfaction'] # a lot of missing values! over 114k! better we do the this calculation seperated

df2 = train_preprocessed.copy()

imputer = IterativeImputer(
    estimator=RandomForestClassifier(n_estimators=50, max_depth=10, n_jobs=-1, random_state=42),
    max_iter=200,
    random_state=42,
    min_value=0,
    imputation_order="ascending" 
)

df_subset = df2[columns_to_convert]
df_imputed_values = imputer.fit_transform(df_subset)
df2[columns_to_convert] = np.round(df_imputed_values).astype(int)

print(imputer.n_iter_)
show_general_stats(df2)

0


### General Stats

,Work Pressure,Study Satisfaction,Job Satisfaction,Work/Study Hours,Financial Stress,Depression
count,143256.000000,143256.000000,143256.000000,143256.000000,143256.000000,143256.000000
mean,3.031049,2.989543,2.850561,6.248597,2.988657,0.181647
std,1.420301,0.606115,1.422842,3.852275,1.413677,0.385555
min,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000
25%,2.000000,3.000000,2.000000,3.000000,2.000000,0.000000
50%,3.000000,3.000000,3.000000,6.000000,3.000000,0.000000
75%,4.000000,3.000000,4.000000,10.000000,4.000000,0.000000
max,5.000000,5.000000,5.000000,12.000000,5.000000,1.000000


### Data Types

Working Professional or Student          object
Work Pressure                             int64
Study Satisfaction                        int64
Job Satisfaction                          int64
Degree                                   object
Have you ever had suicidal thoughts ?    object
Work/Study Hours                          int64
Financial Stress                          int64
Depression                                int64
dtype: object

...

### Tail

,Working Professional or Student,Work Pressure,Study Satisfaction,Job Satisfaction,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Depression
143156,Working Professional,3,3,3,BSc,No,10,5,0
143157,Working Professional,1,3,5,BSc,No,5,4,0
143158,Working Professional,2,3,4,M.Pharm,Yes,1,4,0
143159,Working Professional,4,3,1,Other,Yes,2,5,1
143160,Student,2,3,5,Class 12,Yes,6,5,1
...,...,...,...,...,...,...,...,...,...
143251,Working Professional,1,3,5,BBA,Yes,12,3,0
143252,Working Professional,3,3,1,Other,Yes,8,3,1
143253,Working Professional,1,3,4,Other,Yes,4,4,0
143254,Working Professional,2,3,3,BBA,No,4,5,0


### Sample

,Working Professional or Student,Work Pressure,Study Satisfaction,Job Satisfaction,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Depression
132043,Working Professional,1,3,3,BCA,No,3,5,0
82777,Working Professional,2,3,5,BBA,No,3,4,0
135510,Working Professional,5,3,1,Other,No,0,3,0
16416,Student,3,4,2,Class 12,Yes,6,2,1
38949,Student,2,4,5,Class 12,Yes,6,5,1
...,...,...,...,...,...,...,...,...,...
60101,Working Professional,3,3,5,Other,No,3,4,0
18341,Working Professional,3,3,1,M.Pharm,No,0,2,0
63130,Working Professional,2,3,4,BSc,No,9,5,0
23598,Student,5,3,1,Class 12,Yes,10,5,1


### 

# Scale Numerical features
not needed, there all between 1-5 or 1-10

# Save preprocessed dataframe

In [254]:
# if KAGGLE_ENV:
#     train.to_csv('/kaggle/working/s4-e11-train-concat-imputed.csv', index=False)
# else:
#     train.to_csv( '../kaggle/working/' + '/s4-e11-train-concat-imputed.csv', index=False)

## Fix categorical missing data
There are not any missing data for categorical features.

# Scale Categorical Features
There are different categories at this dataframe.
Working Professional or Student, it is just 1 or 0, so we use OneHotEncoder
Have you ever had suicidal thoughts?, yes/no, already binar
Degree, should use a OrdinalEncoder

In [255]:
df2.head(100)

,Working Professional or Student,Work Pressure,Study Satisfaction,Job Satisfaction,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Depression
0,Working Professional,5,3,2,Other,No,1,2,0
1,Working Professional,4,3,3,Other,Yes,7,3,1
2,Student,2,2,5,B.Pharm,Yes,3,1,1
3,Working Professional,5,3,1,BBA,Yes,10,1,1
4,Working Professional,1,3,1,BBA,Yes,9,4,0
...,...,...,...,...,...,...,...,...,...
95,Working Professional,2,3,5,B.Ed,Yes,10,4,0
96,Working Professional,4,3,3,B.Ed,Yes,4,2,0
97,Working Professional,1,3,3,M.Tech,No,1,2,0
98,Working Professional,2,3,2,B.Ed,No,9,2,0


In [256]:
# Check the unique values in the column 'Degree'
num_unique_degrees = df2["Degree"].nunique()
unique_degrees = df2["Degree"].unique()

print(f"Number of unique categories in the Degree column': {num_unique_degrees}")
print(f"Unique categories in the 'Degree' column': {unique_degrees}")

Number of unique categories in the Degree column': 16
Unique categories in the 'Degree' column': ['Other' 'B.Pharm' 'BBA' 'MCA' 'BSc' 'B.Arch' 'BCA' 'B.Ed' 'B.Com'
 'Class 12' 'M.Tech' 'M.Ed' 'MSc' 'B.Tech' 'LLM' 'M.Pharm']


In [257]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

df = df2.copy()

# Binary Encoding for "Have you ever had suicidal thoughts ?"
df['Have you ever had suicidal thoughts ?'] = df['Have you ever had suicidal thoughts ?'].map({'Yes': 1, 'No': 0})

# One-Hot - Encoding for "Working Professional or Student"
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
encoded_feature = encoder.fit_transform(df[['Working Professional or Student']])
df_encoded = pd.DataFrame(encoded_feature, columns=encoder.get_feature_names_out(['Working Professional or Student']))

df = df.drop(columns=['Working Professional or Student'])
df = pd.concat([df, df_encoded], axis=1)

# One-Hot-Encoding for "Degree"
encoder_degree = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
degree_encoded = encoder_degree.fit_transform(df2[["Degree"]])
degree_columns = encoder_degree.get_feature_names_out(["Degree"])
df_degree = pd.DataFrame(degree_encoded, columns=degree_columns, index=df2.index)

# Final DataFrame
train_final = df.drop(columns=["Degree"], errors='ignore').join(df_degree)

# Show stats
print("OneHotEncoded Degree Data")
show_general_stats(train_final)

OneHotEncoded Degree Data


### General Stats

,Work Pressure,Study Satisfaction,Job Satisfaction,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Depression,Working Professional or Student_Student,Working Professional or Student_Working Professional,Degree_B.Arch,...,Degree_BCA,Degree_BSc,Degree_Class 12,Degree_LLM,Degree_M.Ed,Degree_M.Pharm,Degree_M.Tech,Degree_MCA,Degree_MSc,Degree_Other
count,143256.000000,143256.000000,143256.000000,143256.000000,143256.000000,143256.000000,143256.000000,143256.000000,143256.000000,143256.000000,...,143256.000000,143256.000000,143256.000000,143256.000000,143256.000000,143256.000000,143256.000000,143256.000000,143256.000000,143256.000000
mean,3.031049,2.989543,2.850561,0.494297,6.248597,2.988657,0.181647,0.198267,0.801733,0.061645,...,0.040780,0.035747,0.104736,0.033025,0.040131,0.032264,0.031831,0.037290,0.034721,0.299373
std,1.420301,0.606115,1.422842,0.499969,3.852275,1.413677,0.385555,0.398696,0.398696,0.240510,...,0.197781,0.185660,0.306213,0.178702,0.196267,0.176701,0.175551,0.189472,0.183073,0.457985
min,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,3.000000,2.000000,0.000000,3.000000,2.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,3.000000,3.000000,0.000000,6.000000,3.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.000000,3.000000,4.000000,1.000000,10.000000,4.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,5.000000,5.000000,5.000000,1.000000,12.000000,5.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Data Types

Work Pressure                                             int64
Study Satisfaction                                        int64
Job Satisfaction                                          int64
Have you ever had suicidal thoughts ?                     int64
Work/Study Hours                                          int64
Financial Stress                                          int64
Depression                                                int64
Working Professional or Student_Student                 float64
Working Professional or Student_Working Professional    float64
Degree_B.Arch                                           float64
Degree_B.Com                                            float64
Degree_B.Ed                                             float64
Degree_B.Pharm                                          float64
Degree_B.Tech                                           float64
Degree_BBA                                              float64
Degree_BCA                              

...

### Tail

,Work Pressure,Study Satisfaction,Job Satisfaction,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Depression,Working Professional or Student_Student,Working Professional or Student_Working Professional,Degree_B.Arch,...,Degree_BCA,Degree_BSc,Degree_Class 12,Degree_LLM,Degree_M.Ed,Degree_M.Pharm,Degree_M.Tech,Degree_MCA,Degree_MSc,Degree_Other
143156,3,3,3,0,10,5,0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143157,1,3,5,0,5,4,0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143158,2,3,4,1,1,4,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
143159,4,3,1,1,2,5,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
143160,2,3,5,1,6,5,1,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143251,1,3,5,1,12,3,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143252,3,3,1,1,8,3,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
143253,1,3,4,1,4,4,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
143254,2,3,3,0,4,5,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Sample

,Work Pressure,Study Satisfaction,Job Satisfaction,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Depression,Working Professional or Student_Student,Working Professional or Student_Working Professional,Degree_B.Arch,...,Degree_BCA,Degree_BSc,Degree_Class 12,Degree_LLM,Degree_M.Ed,Degree_M.Pharm,Degree_M.Tech,Degree_MCA,Degree_MSc,Degree_Other
83839,2,3,2,0,3,2,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5072,4,3,2,1,0,3,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
65554,2,3,1,0,2,2,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
130239,4,3,1,1,11,4,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24620,1,3,5,1,8,5,0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102910,2,3,5,0,3,1,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
142198,1,3,4,0,0,1,0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
131889,4,3,1,0,4,4,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104307,4,3,2,0,4,1,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### 

# Save preprocessed final dataframe

In [ ]:
# if KAGGLE_ENV:
#     train_final.to_csv('/kaggle/working/s4-e11-train-concat-final.csv', index=False)
# else:
#     train_final.to_csv( '../kaggle/working/' + '/s4-e11-train-concat-final.csv', index=False)

# Feature Selection (optional)
Delete attributes there aren't providing any information (correlation matrix).

# Create new Features (optional)
- Discretizing Continuous Features
- Decomposing Features and Date/Time Variables
- Aggregate attributes to better new attributes